<a href="https://colab.research.google.com/github/Pruthviraj141/RAG-Based-Models-/blob/main/Copy_of_ZERO_Cost_RAG_model_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q \
  langchain \
  langchain-core \
  langchain-community \
  langchain-chroma \
  langchain-openai \
  chromadb \
  pypdf


In [ ]:
import os
import warnings
warnings.filterwarnings("ignore")

# 🔑 OpenRouter API Key (Free of costtttt)
os.environ["OPENAI_API_KEY"] = "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"

# 🔒 Disable LangSmith completely (IMPORTANT)
os.environ["LANGCHAIN_TRACING_V2"] = "false"


In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="google/gemma-3-27b-it:free",
    base_url="https://openrouter.ai/api/v1",
    temperature=0.2,
    default_headers={
        "HTTP-Referer": "https://your-site.com",   # optional
        "X-Title": "Hybrid PDF RAG",               # optional
    },
)


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)


/tmp/ipython-input-3055314890.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
files.upload()


Saving pdf.pdf to pdf.pdf


{'pdf.pdf': b'%PDF-1.4\n3 0 obj\n<</Type /Page\n/Parent 1 0 R\n/MediaBox [0 0 595.32 841.92]\n/Resources 2 0 R\n/Contents 6 0 R>>\nendobj\n6 0 obj\n<</Filter /FlateDecode /Length 71>>\nstream\nx\x9c3R\xf0\xe22\xd035W(\xe7*T0P\xf0R0T(\x07\xd2Y@\xec\x0e\xc4\xe9@QC=\x03 P\x00A\x18\x13\x85J\xce\xe5\xd2\x0f\t\xf01Tp\xc9W\x08\xe4\n\xe4\x02+R@&\x8b\xd2\xb9\x006f\x14\xa5\nendstream\nendobj\n4 0 obj\n<</Type /Page\n/Parent 1 0 R\n/MediaBox [0 0 595.32 841.92]\n/Resources 2 0 R\n/Contents 7 0 R>>\nendobj\n7 0 obj\n<</Filter /FlateDecode /Length 73>>\nstream\nx\x9c3R\xf0\xe22\xd035W(\xe72\xd4300P@&\x8b\xd2\xb9\n\x15\x0c\x14\xbc\x14\x0c\x15\xca\x81t\x16\x10\xbb\x031H\x14\xac\xc0@\x01\x04aL\x14*9\x97K?$\xc0\xc7H\xc1%_!\x90+\x10\xbb\xe1\x00\x06C\x18\xb6\nendstream\nendobj\n5 0 obj\n<</Type /Page\n/Parent 1 0 R\n/MediaBox [0 0 595.32 841.92]\n/Resources 2 0 R\n/Contents 8 0 R>>\nendobj\n8 0 obj\n<</Filter /FlateDecode /Length 73>>\nstream\nx\x9c3R\xf0\xe22\xd035W(\xe72\xd4300P@&\x8b\xd2\xb9\n\x15\x0c

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/pdf.pdf")
documents = loader.load()


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=150,
)

splits = text_splitter.split_documents(documents)


In [ ]:
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="./chroma_pdf"
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4})


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
"""
You are an AI assistant using a PDF as the primary knowledge source.

Rules:
1. First check the PDF context.
2. If the answer exists in the PDF:
   - Answer ONLY using the PDF.
3. If the answer is NOT in the PDF:
   - Say exactly:
     "Not found in the PDF.
      Using general knowledge, the answer is:"
   - Then answer using your own knowledge.
4. Keep the answer concise (max 5 sentences).

PDF Context:
{context}

Question:
{input}
"""
)


In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

def format_docs(docs):
    if not docs:
        return ""
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {
        "context": retriever | format_docs,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)


In [ ]:
rag_chain.invoke("Explain the main topic of this document")


'\nThe document is about PICT InC 2025, a flagship international-level technical event organized by Pune Institute of Computer Technology (PICT). It provides information about the event’s purpose, which is to help students turn ideas into impactful solutions and encourage innovation. The document details the events under InC 2025 – Impetus, Concepts, Pradnya, TechFiesta, and a special event. It also includes information about the total prize pool of ₹7,00,000+ and specific details for the Impetus event, like eligibility and registration fees.'

In [ ]:
rag_chain.invoke("Provide me the contact Number of more details real quick")


'\nGauri Desale – 8669178247.'

In [ ]:
rag_chain.invoke("Who is Current President of Zimbabwe ")


'Not found in the PDF.\nUsing general knowledge, the answer is:\nThe current president of Zimbabwe is Emmerson Mnangagwa. He has been in office since November 2017. Mnangagwa is a member of the ZANU-PF party, which has been the ruling party in Zimbabwe since its independence in 1980. His presidency has been marked by economic challenges and political tensions.'